In [1]:
import ipywidgets
import plotly
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive,interact, HBox, Layout,VBox
import pandas as pd
import numpy as np

In [2]:
def QF_sims(Smoothing=0.8, Stochastic = 1):
    
    df = pd.read_excel("/Users/Khalifa/Desktop/EDHEC/data/QF scenarios data.xlsx").set_index("Years")
    df = df["Portfolio"]
    
    mu = 0.06
    sigma = 0.125
    start_price = 100
    returns = np.random.normal(loc=mu, scale=sigma, size=32)
    price = start_price*(1+returns).cumprod()
    market_value= []
    strict =[]
    fixed =[]
    yale=[]
    capital = 125
    s = 5
    dist_rate = 0.04
    
    
    if Stochastic == 1:
        
        Stochastic = returns
    else:
        
        Stochastic = df
    
    
    for i in Stochastic:
    
        capital *= (1+i)
        s *= 1.02
        f = dist_rate * capital
        y = (s) + Smoothing*(dist_rate*capital-s)
        capital -= y
        market_value.append(capital)
        strict.append(s)
        fixed.append(f)
        yale.append(y)
        
    a = pd.DataFrame(strict, columns=["Strict"])
    b =pd.DataFrame(fixed, columns=["Fixed"])
    c = pd.DataFrame(market_value, columns=["MV"])
    d = pd.DataFrame(yale, columns=["Yale"])
    sim =a.join([b,c,d])
    rets = pd.DataFrame((1+returns).cumprod(), index= sim.index, columns = ["Returns"])
    
    years = pd.DataFrame(list(range(1,33)))
    real_spending =[]
    real_mv = []
    
    for i, rows in years.iterrows():
    
        y= d["Yale"][i]
        z = years[0][i]
        real_s = y/(1.02**z)
        real_mv.append((c["MV"][i]/(1.02**z)))
        real_spending.append(real_s)
        
    r_mv = pd.DataFrame(real_mv, index = sim.index, columns = ["Real Market Value"])
    r_spend = pd.DataFrame(real_spending, index = sim.index, columns = ["Real Spending"])
    
    budget = []
    start = 5
    for i in range(0,32):
    
        start *=1.02
        budget.append(start)
    
    mv_budget = []
    start_mv = 125
    
    for i in range(0,32):
    
        start_mv *=1.02
        mv_budget.append(start_mv)
        
    mv_budget = pd.DataFrame(mv_budget, index = sim.index, columns = ["Inflation"])
    
    budget = pd.DataFrame(budget, index = sim.index, columns = ["Budget"])
    final = sim.join([r_mv,r_spend, rets, budget, mv_budget])
    final["Delta"] = final["Yale"] - final["Budget"]
    
    data = {'Strict':5, 'Fixed':5, 'MV':125,'Yale':5, 'Real Market Value':125, 'Real Spending':5,'Returns':1, 
            'Budget':5, 'Inflation':125, 'Delta':0 } 
    
    final = pd.concat([pd.DataFrame(data, index = [0]),final])
    final = final.reset_index()
    final = final.sort_index()
    final = final.drop(["index"], axis=1)
    
    
    fig = make_subplots(rows=2, cols=2,subplot_titles=("Market Value of Assets", "Distributions vs Budget", "Delta", "Portfolio Returns"),horizontal_spacing = 0.03, vertical_spacing=0.09)
    
    
    fig.add_trace(go.Scatter(x = final.index,y=final["MV"],
                        mode='lines',
                        name='Market Value', line_color="maroon", legendgroup="1"), row=1,col=1)
    fig.add_trace(go.Scatter(x = final.index,y=final["Inflation"],
                        mode='lines',
                        name='Inflation', line_color="grey",legendgroup="1"),row=1,col=1)
    #fig.add_trace(go.Scatter(x = final.index,y=final["Fixed"],
                        #mode='lines',
                        #name='Fixed Amount', line_color="darkblue",legendgroup="1"),row=1,col=1)
    
    fig.add_trace(go.Scatter(x = final.index,y=final["Yale"],
                        mode='lines',
                        name='Distributions', line_color="violet",legendgroup="2"),row=1,col=2)
    fig.add_trace(go.Scatter(x = final.index,y=final["Budget"],
                        mode='lines',
                        name='Budget', line_color="green",legendgroup="2"),row=1,col=2)
    #fig.add_trace(go.Scatter(x = comparison.index,y=mv["MV"],
                        #mode='lines',
                        #name='Portfolio', line_color="red",legendgroup="2"),row=1,col=2)
    
    fig.add_trace(go.Bar(x = final.index ,y=final["Delta"],legendgroup="3", name="Delta"), row=2, col=1)
    #fig.add_trace(go.Bar(y=bonds_annualized_rets,legendgroup="3", name="Fixed Income"), row=2, col=1)
    
    fig.add_trace(go.Scatter(x = final.index,y=final["Returns"],
                        mode='lines',
                        name='Returns', line_color="red",legendgroup="4"),row=2,col=2)
    
    # Update xaxis properties
    #fig.update_xaxes(title_text="Returns", showgrid=False, row=2, col=1)
    #fig.update_xaxes(title_text="Dates", type="log", row=2, col=2)

    # Update yaxis properties
    #fig.update_yaxes(title_text="Distributions", row=1, col=1)
    #fig.update_yaxes(title_text="Returns", range=[40, 80], row=2, col=1)
    #fig.update_yaxes(title_text="yaxis 3 title", showgrid=False, row=2, col=1)
    
    fig.update_layout(height=700, width=1500,legend_tracegroupgap = 100)
    
    return fig

In [4]:
widget= interact(QF_sims, Smoothing = (0,1,0.1), Stochastic = (0,1,1))
widget

interactive(children=(FloatSlider(value=0.8, description='Smoothing', max=1.0), IntSlider(value=1, description…

<function __main__.QF_sims(Smoothing=0.8, Stochastic=1)>